## Langchain Documents and Vector DB - sentence-transformers, ChromaDB

1. VECTOR DB
- https://huggingface.co/sentence-transformers
- https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

2. LANGCHAIN 手冊
https://python.langchain.com/docs/integrations/vectorstores

3. 學習LANGCHAIN -> Documents and Vector DB 

## 初始環境設定

In [ ]:
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
current_foldr=!pwd
current_foldr=current_foldr[0]
current_foldr

## 安裝套件

In [ ]:
## For colab
!pip install chromadb gdown langchain langchain-community openai pypdf python-dotenv sentence-transformers -q

### LOAD LIBRARY### 2. LOAD LIBRARY

In [1]:
# Load library
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

### 文件處理

In [2]:
!mkdir -p data/pdf/
!gdown 1AldhEWVCtcE50XARgSnXR0azZ965nNmT -O data/pdf/

Downloading...
From: https://drive.google.com/uc?id=1AldhEWVCtcE50XARgSnXR0azZ965nNmT
To: /work/c00cjz00/github/LLAMA-on-HPC/class_20240704/class/data/pdf/e2729e76-29a0-4be5-9eef-67809b05d6b9.pdf
100%|████████████████████████████████████████| 305k/305k [00:00<00:00, 27.0MB/s]


In [3]:
# 文件入庫
pdf_file='./data/pdf/e2729e76-29a0-4be5-9eef-67809b05d6b9.pdf'
loader= PyPDFLoader(pdf_file)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

print(len(texts))
print(texts[3:4])

8
[Document(page_content='3 \n 定義資訊安全管理系統 (ISMS)的需求標準，包含建立、實施\n、檢查、改進等層面 。透過嚴謹、具有高度公信力的稽核單\n位，確保組織採用系統化的作業流程來管理組織內部及客戶\n個人資料的安全。  \n2. ISO 27001 對本公司的意義  \n藉由導入 ISO 27001 資安管理機制，證明 本公司經營團隊具\n有完善的資訊安全管理體系， 其涵蓋範圍包括 資訊安全政策\n、資訊安全組織、人力資源安全、資產管理、存取控制、密\n碼學、實體及環境安全、運作安全、通訊安全、系統獲取開\n發與維護 、供應者關係、資訊安全事故管理、營運持續管理\n、遵循性 等作業； 透過健全的資安管理體系，除能有效 對用\n戶資訊安全提供保障，也進階提升團隊競爭力，達到永續經\n營的目標。  \n三、 人力資源安全 [A7] \n本公司對甄選及晉用之人員均會檢核與確認其所 學、經歷及專業資格 ，\n負責重要業務活動前，會針對該項活動所需之技能 、人員的資格、 適任\n與否予以評估。  \n新進人員到職後，由財管 部及相關部門 實施新進人員訓練，使其了解本\n公司組織、管理規章及安全事項。  \n另有專責部門辦理定期或不定期之資訊安全教育訓練及宣導，促使所有\n人員瞭解資訊安全之重要性及各種可能的安全風險，並說明違反資訊安\n全規定時可能招致的處罰及法律責任，以提高本公司人員資訊安全意識\n，促其遵守資訊安全規定 ；如人員因 工作職責須使用機敏性資訊 或/及\n設施，需經過授權核准使用 。 \n所有人員須依相關法令及本公司規範負保密責任，並簽訂保密切結書。  \n四、 資產管理 [A8] \n本公司採取嚴謹的風險評鑑方法論進行預測，與風險處理，以確保資料\n資產安全。  \n每年均進行資產盤點 活動，將資產類別、資產名稱、型號、說明、放置\n地點、數量、資產擁有者、 風險管理者、 資產管理者及資產使用者等項\n目納入並執行風險評鑑與風險處理。  \n五、 存取控制 [A9] \n本公司採取嚴謹的控制措施避免未經授權的存取，以 保護客戶和自身的\n資料資產安全 。 \n權限控管與識別特 殊權限是存取控制的基本原則， 本公司每位員工 皆擁\n有唯一的帳號和 密碼，並依據其角色職責賦予最小化之權限。  \n如需特殊權限存取相關的資源

### 片段文字向量化與暫時存入資料庫

In [4]:
Embeddings_ID="sentence-transformers/all-MiniLM-L6-v2"
embeddings=HuggingFaceEmbeddings(model_name=Embeddings_ID)
vectortdb = Chroma.from_documents(texts, embeddings)
#DB_PATH = 'vectorstore/db_chroma'
#vectortdb = Chroma.from_documents(documents=texts, embedding=embeddings, persist_directory=DB_PATH)

/home/c00cjz00/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/c00cjz00/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


in oss file


/home/c00cjz00/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### 輸入文字像量化與暫存資料庫搜尋

In [ ]:
# Load DB
#Embeddings_ID="sentence-transformers/all-MiniLM-L6-v2"
#embeddings=HuggingFaceEmbeddings(model_name=Embeddings_ID)
#DB_PATH = 'vectorstore/db_chroma'
#vectortdb = Chroma(persist_directory=DB_PATH, embedding_function=embeddings)

import os
#: Test Search in Vector DB
query = "請說明本季季報內容？請依以下順序描述重點：收入、毛利率、營運支出、營運利潤率、淨利潤和每股盈餘"
source_documents=vectortdb.similarity_search(query, k=3)

for i, doc in enumerate(source_documents):
    page_content=source_documents[i].page_content
    page=source_documents[i].metadata["page"]
    source=source_documents[i].metadata["source"]
    file = os.path.basename(source) 
    print("Source: "+file+", Page "+str(page+1) )
    print(page_content)
    print("\n\n")

In [11]:
page_content=source_documents[2].page_content

In [12]:
page_content

'2 \n 壹、前言 \n德欣寰宇 股份有限公司（以下簡稱 本公司）致力於 系統工程整合服務 的研發\n、營運與行銷，以 客戶需求提供專業化、個人化、創新化、品牌化的安全整合解\n決方案及最佳市場訊息 ，並因應當前網際網路的蓬勃發展趨勢規劃實用的產品 ，\n落實「客戶 滿意度即為品質」之服務理念。  \n貳、範圍 \n本資訊安全 白皮書將介紹 本公司在資訊安全管理上的作法，具體涵蓋 資訊安\n全政策、資訊安全組織、人力資源安全、資產管理、存取控制、密碼學、實體及\n環境安全、運作安全、通訊安全、 系統獲取開發與維護、供應者關係、資訊安全\n事故管理、營運持續管理、遵循性等 14個領域。  \n參、資訊安全管理架構定義  \n一、資訊安全政策 [A5] \n本公司服務之資訊安全目標為：建置符合國際標準所要求之資訊安全管\n理系統 (Information Security Management System )，以確保 顧問服務\n如ISO 27001 、ISO 20000 、ISO 29100 、BS 10012 等驗證輔導，及資\n安技術服務如 SOC監控服務、資安健診 、社交工程 、弱點掃描及滲透\n測試等之 機 密 性 (Confidentiality) 、 完 整 性 (Integrity) 、 可 用 性\n(Availability) 及適法性 (Compliance) 。 \n本公司服務營運以確保資訊安全為前提，而達成資訊安全的目標則是全\n體公司同仁的責任 ，提供無虞之服務 亦為本公司 的決心與承諾 。 \n二、 資訊安全組織 [A6] \n(一)、 資訊安全團隊  \n本公司為了提供國內外企業 (電信、醫療 、證券金融業 等)、公私\n立學校及政府機關 (構)最安全完善的 服務，成立了資訊安全管理\n團隊，團隊組成 由專案管理部、技術部、顧問部 等團隊所組成 ，\n團隊成員的專業能力分別 有：資訊安全管理 、個資安全管理、第\n三方支付安全管理、合規安全管理、 系統安全 管理、事件回應管\n理、網路安全 管理及數位鑑識管理 。 \n(二)、 安全稽核 組 \n安全稽核 組是本公司資訊安全組織，負責維護資訊安全的有效性\n，其中包含了：控制環境、風險評估、控制作業、資訊溝通及監\n督，透過稽核以滿足 法令法規及內部程序之 遵循性要求。 \n(三)、 